In [1]:
import tensorflow as tf
import numpy as np
import os
import sys

In [2]:
dir_actual    = os.path.abspath('')
dir_parent = os.path.dirname(dir_actual)
if not dir_parent in sys.path: sys.path.append(dir_parent)

In [3]:
tf.random.set_seed(1234)
np.random.seed(1234)

## Loading CIFAR10 Data

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# Normalizando
x_train = x_train/255.
x_test  = x_test/255.
# Reduciendo la dimension innecesaria
y_train = np.squeeze(y_train)
y_test  = np.squeeze(y_test)
# Creando subconjunto de validacion
x_valid = x_train[40000:]  
y_valid = y_train[40000:]  

x_train = x_train[:40000]
y_train = y_train[:40000]

## Creating DataSet Objects

In [6]:
from functions import to_categorical, AlexNet_resize

bs = 32

# Training
#-----------------------------------------
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0])
train_dataset = train_dataset.map(to_categorical)
train_dataset = train_dataset.map(AlexNet_resize)
# Divide in batches
train_dataset = train_dataset.batch(bs)
# Repeat
train_dataset = train_dataset.repeat()

#Validation   
# -----------------------
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
# Enconding
valid_dataset = valid_dataset.map(to_categorical)
# Resize
valid_dataset = valid_dataset.map(AlexNet_resize)
# Divide in batches
valid_dataset = valid_dataset.batch(bs)
# Repeat
valid_dataset = valid_dataset.repeat()

#Testing 
# -------------------
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(to_categorical)
test_dataset = test_dataset.map(AlexNet_resize)
test_dataset = test_dataset.batch(1)

## Building Model

In [8]:
from models import AlexNet_body

input_layer   = tf.keras.Input((227,227,3))
output_layer  = AlexNet_body(input_layer, Quantization = False)

AlexNet  = tf.keras.Model(inputs=input_layer, outputs=output_layer)
AlexNet.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
lambda (Lambda)              (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
lambda_1 (Lambda)            (None, 55, 55, 96)        0         
_________________________________________________________________
tf_op_layer_Relu (TensorFlow [(None, 55, 55, 96)]      0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384   

## Training Options

In [9]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
AlexNet.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Callbacks

In [10]:
import os
from datetime import datetime

early_stop = True
tensorboard = False
checkpoints = False


# Save Directory (Change for your own directory)
cwd = os.getcwd()

if (tensorboard or checkpoints):
# Creating SubDirectory
    exps_dir = os.path.join(cwd, 'GraphData')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)
    now = datetime.now().strftime('%b%d_%H-%M-%S')
    exp_name = "AlexNet"
    exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
if checkpoints:
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                       save_weights_only=True)  # False to save the model directly
    callbacks.append(ckpt_callback)

# ----------------

# Visualize Learning on Tensorboard
# ---------------------------------
if tensorboard:
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,profile_batch=0,histogram_freq=1,
                                                 write_graph=True,write_images=True,embeddings_freq=1)  
    callbacks.append(tb_callback)

# Early Stopping
# --------------
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    callbacks.append(es_callback)

# ---------------------------------

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR c--port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

## Training

In [ ]:
AlexNet.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=int(np.ceil(x_train.shape[0] / bs)),
          validation_data=valid_dataset,
          validation_steps=int(np.ceil(x_valid.shape[0] / bs)), 
          callbacks=callbacks)

## Saving Weights

In [ ]:
Wgt_dir = os.path.join(cwd, 'TrainedWeights')
Wgt_dir = os.path.join(Wgt_dir, 'Weights')
AlexNet.save_weights(Wgt_dir)